# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Lee  ->  K. Lee  |  ['K. Lee']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Li  ->  J. Li  |  ['J. Li']


M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
Arxiv has 104 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.08048


extracting tarball to tmp_2507.08048...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.08201
extracting tarball to tmp_2507.08201... done.
Retrieving document from  https://arxiv.org/e-print/2507.08257


extracting tarball to tmp_2507.08257...

...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'main.bbl' from 'tmp_2507.08561/main.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


 done.
Retrieving document from  https://arxiv.org/e-print/2507.08797


extracting tarball to tmp_2507.08797... done.


F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']


Found 261 bibliographic references in tmp_2507.08797/main.bbl.
Issues with the citations
syntax error in line 1432: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08545-b31b1b.svg)](https://arxiv.org/abs/2507.08545) | **Euclid: Early Release Observations. A combined strong and weak lensing solution for Abell 2390 beyond its virial radius**  |
|| J. M. Diego, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-14*|
|*Comments*| *13 pages, 8 figures*|
|**Abstract**|            Euclid is presently mapping the distribution of matter in the Universe in detail via the weak lensing (WL) signature of billions of distant galaxies. The WL signal is most prominent around galaxy clusters, and can extend up to distances well beyond their virial radius, thus constraining their total mass. Near the centre of clusters, where contamination by member galaxies is an issue, the WL data can be complemented with strong lensing (SL) data which can diminish the uncertainty due to the mass-sheet degeneracy and provide high-resolution information about the distribution of matter in the centre of clusters. Here we present a joint SL and WL analysis of the Euclid Early Release Observations of the cluster Abell 2390 at z=0.228. Thanks to Euclid's wide field of view of 0.5 deg$^$2, combined with its angular resolution in the visible band of 0."13 and sampling of 0."1 per pixel, we constrain the density profile in a wide range of radii, 30 kpc < r < 2000 kpc, from the inner region near the brightest cluster galaxy to beyond the virial radius of the cluster. We find consistency with earlier X-ray results based on assumptions of hydrostatic equilibrium, thus indirectly confirming the nearly relaxed state of this cluster. We also find consistency with previous results based on weak lensing data and ground-based observations of this cluster. From the combined SL+WL profile, we derive the values of the viral mass $M_{200} = (1.48 \pm 0.29)\times10^{15}\, \Msun$, and virial radius $r_{200} =(2.05\pm0.13 \, {\rm Mpc}$), with error bars representing one standard deviation. The profile is well described by an NFW model with concentration c=6.5 and a small-scale radius of 230 kpc in the 30\,kpc $< r <$ 2000\,kpc range that is best constrained by SL and WL data. Abell 2390 is the first of many examples where Euclid data will play a crucial role in providing masses for clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08797-b31b1b.svg)](https://arxiv.org/abs/2507.08797) | **Multi-frequency analysis of the ALMA and VLA high resolution continuum observations of the substructured disc around CI Tau. Preference for sub-mm-sized low-porosity amorphous carbonaceous grains**  |
|| <mark>F. Zagaria</mark>, et al. |
|*Appeared on*| *2025-07-14*|
|*Comments*| *43 pages, 23 figures, 6 tables. Accepted for publication on A&A*|
|**Abstract**|            (Abridged) We present high angular resolution and sensitivity ALMA 3.1 mm and VLA 9.1 mm observations of the disc around CI Tau. These new data were combined with similar-resolution archival ALMA 0.9 and 1.3 mm observations and new and archival VLA 7.1 mm, 2.0, 3.0, and 6.0 cm photometry to study the properties of dust in this system. At wavelengths <3.1 mm, CI Tau's continuum emission is very extended and highly substructured (with three gaps, four rings, and two additional gap-ring pairs identified by non-parametric visibility modelling). Instead, the VLA 9.1 mm data are dominated by a bright central component, only partially (< 50%) due to dust emission, surrounded by a marginally detected, faint, and smooth halo. We fitted the ALMA and VLA 9.1 mm data together, adopting a physical model that accounts for the effects of dust absorption and scattering. For our fiducial dust composition ("Ricci" opacities), we retrieved a flat maximum grain size distribution across the disc radius of $(7.1\pm0.8)\times10^{-2}$ cm, that we tentatively attributed to fragmentation of fragile dust or bouncing. We tested, for the first time, the dependence of our results on the adopted dust composition model to assess which mixture can best reproduce the observations. We found that the "Ricci" opacities work better than the traditionally adopted "DSHARP" ones, while graphite-rich mixtures perform significantly worse. We also show that, for our fiducial composition, the data prefer low-porosity (< 70%) grains, in contrast with claims of highly porous aggregates in younger sources, which we tentatively justified by time-dependent compaction. Our results are in line with constraints from disc population synthesis models and naturally arise from CI Tau's peculiar spectral behaviour, making this disc an ideal target for deeper cm-wavelength and dust polarisation follow-ups.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08257-b31b1b.svg)](https://arxiv.org/abs/2507.08257) | **SDSS-V LVM: Collisionless Shocks in the Supernova Remnant RCW86**  |
|| S. K. Sarbadhicary, et al. -- incl., <mark>K. Kreckel</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2025-07-14*|
|*Comments*| *30 pages, 16 figures. Submitted to ApJ*|
|**Abstract**|            The supernova remnant (SNR) RCW86 is among the few SNRs with Balmer-emission lines containing broad and narrow spectral components that trace fast, non-radiative shocks in partially-ionized gas.\ These are invaluable laboratories for collisionless shock physics, especially for poorly-understood phenomena like electron-ion equilibration, and shock precursors. Here we present the first $\sim$0.3 pc spatial scale integral field unit (IFU) observations of the southwestern RCW86 shock, obtained as part of the Sloan Digital Sky Survey-V Local Volume Mapper (SDSS-V LVM). The forward shock, clearly visible as thin filaments in narrowband images, have broad H$\alpha$ components, indicating shock velocities varying from 500--900 km/s in the south to 1000--1500 km/s in the north. The varying velocity widths and broad-to-narrow intensity ratios show that electrons and ions have lower equilibration ($T_e/T_p \rightarrow 0.1$) in faster ($>$800 km/s) shocks, in line with previous studies. The broad components are generally redshifted from the narrow components by $\lesssim$100 km/s, likely due to shock-obliquity or non-Maxwellian post-shock distributions. We observe high extinction-corrected Balmer-decrements of 3--5 in the narrow components, indicating that conversion of Ly$\beta$ photons to H$\alpha$ is more efficient than Ly$\gamma$ to H$\beta$. Broad HeII$\lambda$4686 was marginally ($\gtrsim$2$\sigma$) detected in the southern shock, meaning the shock is impacting gas with high ($>$30--100\%) neutral fraction. We also find the first evidence of an intermediate H$\alpha$ component in RCW86, with $\Delta$V(FWHM) = 193--207 km/s, likely due to a neutral precursor. We also briefly discuss the southwestern radiative shock, and lay out the exciting future of studying astrophysical shocks with LVM.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08048-b31b1b.svg)](https://arxiv.org/abs/2507.08048) | **A candidate field for deep imaging of the Epoch of Reionization observed with MWA**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-07-14*|
|*Comments*| *Accepted for publication in MNRAS; 19 pages, 15 figures, and 11 tables*|
|**Abstract**|            Deep imaging of structures from the Cosmic Dawn (CD) and the Epoch of Reionization (EoR) in five targeted fields is one of the highest priority scientific objectives for the Square Kilometre Array (SKA). Selecting 'quiet' fields, which allow deep imaging, is critical for future SKA CD/EoR observations. Pre-observations using existing radio facilities will help estimate the computational capabilities required for optimal data quality and refine data reduction techniques. In this study, we utilize data from the Murchison Widefield Array (MWA) Phase II extended array for a selected field to study the properties of foregrounds. We conduct deep imaging across two frequency bands: 72-103 MHz and 200-231 MHz. We identify up to 2,576 radio sources within a 5-degree radius of the image center (at RA (J2000) $8^h$ , Dec (J2000) 5°), achieving approximately 80% completeness at 7.7 mJy and 90% at 10.4 mJy for 216 MHz, with a total integration time of 4.43 hours and an average RMS of 1.80 mJy. Additionally, we apply a foreground removal algorithm using Principal Component Analysis (PCA) and calculate the angular power spectra of the residual images. Our results indicate that nearly all resolved radio sources can be successfully removed using PCA, leading to a reduction in foreground power. However, the angular power spectra of the residual map remains over an order of magnitude higher than the theoretically predicted CD/EoR 21 cm signal. Further improvements in data reduction and foreground subtraction techniques will be necessary to enhance these results.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08201-b31b1b.svg)](https://arxiv.org/abs/2507.08201) | **Detecting Intermediate-mass Black Holes Using Miniature Pulsar Timing Arrays in Globular Clusters**  |
|| X. Chen, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2025-07-14*|
|*Comments*| *9 pages, 3 figures, comments welcome!*|
|**Abstract**|            Theoretical models predict that intermediate-mass black holes (IMBHs) exist in globular clusters (GCs), but observational evidence remains elusive. Millisecond pulsars (MSPs), which are abundant in GCs and have served as precise probes for gravitational waves (GWs), offer a unique opportunity to detect potential IMBH binaries in GCs. Here, we consider the possibility of using multiple MSPs in a GC to form a miniature pulsar timing array (PTA), so as to take advantage of their correlated timing residuals to search for potential IMBH binaries in the same cluster. Our semi-analytical calculations reveal that nearby IMBH binaries around MSPs in GCs could induce microsecond-level timing residuals. In GCs like $\omega$ Centauri and M15, favorable configurations are found which could lead to the detection of binaries with mass ratios $q\gtrsim0.1$ and orbital periods of a few days. We estimate that future higher-precision timing programs could achieve $100$-nanosecond sensitivity, substantially expanding the searchable parameter space and establishing mini-PTAs as powerful detectors of IMBHs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08561-b31b1b.svg)](https://arxiv.org/abs/2507.08561) | **Tracing Missing Baryons in the Cosmic Filaments with tSZ and CMB-Lensing Stacking**  |
|| <mark>J. Li</mark>, Y. Zheng, W. Zhu |
|*Appeared on*| *2025-07-14*|
|*Comments*| *28 pages, 14 figures*|
|**Abstract**|            We investigate the distribution of missing baryons in cosmic filaments by stacking approximately 31,300 filaments across the northern and southern SDSS sky regions using Planck Compton-$y$ and CMB lensing maps. Filaments are identified using the DisPerSE algorithm applied to the SDSS LOWZ-CMASS galaxy samples, selecting structures with lengths between 30-100 cMpc and redshifts in the range $0.2 < z < 0.6$. Radial profiles are extracted out to 25 cMpc from the filament spines, and galaxy clusters with halo masses above $3 \times 10^{13}$ solar masses are masked to reduce contamination. We detect the thermal Sunyaev-Zel'dovich (tSZ) signal at $7.82\sigma$ and the CMB lensing signal at $7.78\sigma$. The stacked profiles are corrected by a geometric bias related to filament inclination with respect to the line of sight, and modeled assuming isothermal, cylindrically symmetric gas distributions. We explore different gas and matter density distributions, focusing on $\beta$-models with $(\alpha, \beta) = (2, 2/3)$ or $(1, 1)$. By jointly fitting the Compton-$y$ and lensing convergence profiles, we constrain the central electron overdensity and temperature to be $\delta = 5.90^{+4.18}_{-1.98}$ and $T_e = 2.71^{+0.63}_{-0.51} \times 10^6$ K for the standard $\beta$-model. These results suggest that the filamentary WHIM in our selected long filaments contributes a significant baryon fraction of $0.124^{+0.020}_{-0.021} \times$ {\Omega}_b to the cosmic baryon budget.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08709-b31b1b.svg)](https://arxiv.org/abs/2507.08709) | **Unambiguous GeV gamma-rays from molecular clouds illuminated by particles diffusing from the adjacent supernova remnant G335.2+0.1 that is confined in an expanding bubble**  |
|| C. Huang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-14*|
|*Comments*| *ApJ manuscript after revision incorporating the referee's suggestions. 15 pages. 10 figures & 4 tables included*|
|**Abstract**|            We report the detection of GeV gamma-ray emission likely associated with supernova remnant (SNR) G335.2+0.1 and the finding of a molecular cloud ($\sim20$--$30^\prime$ in angular size) that is very likely in physical contact with the SNR and responsible for the gamma-ray emission. Using the 16.8 yr Fermi-LAT data, an extended emission, with a significance of 13.5 $\sigma$ and a radius 0.24° in 0.2--500 GeV in the uniform-disk model, was found to the adjacent east of the SNR. With archival Mopra CO-line data, a large molecular clump at local-standard-of-rest velocity $\sim-48$ to $-43$ km s$^{-1}$ was revealed appearing coincident with the gamma-ray source. The SNR was found located in a cavity encircled by a 'C'-shaped ring-like molecular shell at $-45$ to $-43$ km s$^{-1}$. This morphological agreement, together with the position-velociy diagrams made along lines cutting across the cavity, suggests that the SNR was evolving in the expanding molecular bubble created by the stellar wind of the progenitor with a mass $\gtrsim 15 M_{\mathrm{sun}}$. The giant molecular cloud, visible at around $-46$ km s$^{-1}$, and the associated SNR are thus estimated to lie at a kinematic distance of 3.1 kpc, with the HI absorption taken into account. We suggest that the SNR has entered the radiative phase after the blastwave recently struck the cavity wall. With the evolutionary scenario of the SNR, we demonstrate that the gamma-ray emission reported here can be naturally interpreted by the hadronic interaction between the accelerated protons that escaped from the SNR shock and the eastern large molecular clump.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.08545/./Figs/Profile_A2390_MsunPc2_Feb2025_V2.png', 'tmp_2507.08545/./Figs/A2390_HST_Euclid_Arcs_CC_Feb2025.png', 'tmp_2507.08545/./Figs/NumberShear_G1_G2_Grid_30arcmin_V5_FINAL.png']
copying  tmp_2507.08545/./Figs/Profile_A2390_MsunPc2_Feb2025_V2.png to _build/html/
copying  tmp_2507.08545/./Figs/A2390_HST_Euclid_Arcs_CC_Feb2025.png to _build/html/
copying  tmp_2507.08545/./Figs/NumberShear_G1_G2_Grid_30arcmin_V5_FINAL.png to _build/html/
exported in  _build/html/2507.08545.md
    + _build/html/tmp_2507.08545/./Figs/Profile_A2390_MsunPc2_Feb2025_V2.png
    + _build/html/tmp_2507.08545/./Figs/A2390_HST_Euclid_Arcs_CC_Feb2025.png
    + _build/html/tmp_2507.08545/./Figs/NumberShear_G1_G2_Grid_30arcmin_V5_FINAL.png
found figures ['tmp_2507.08797/./image_montage.png', 'tmp_2507.08797/./spectral_index.png', 'tmp_2507.08797/./fit_results/results_comparison.png']
copying  tmp_2507.08797/./image_montage.png to _build/html/
copying  tmp_2507.08797/./spectral_index.png t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vect}[1]{\vec{#1}}$
$\newcommand{\Msun}{{  M}_{\odot}}$
$\newcommand{\wslap}{\texttt{WSLAP+} }$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$: Early Release Observations. A combined strong and weak lensing solution for Abell 2390 beyond its virial radius$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.08545-b31b1b.svg)](https://arxiv.org/abs/2507.08545)<mark>Appeared on: 2025-07-14</mark> -  _13 pages, 8 figures_

</div>
<div id="authors">

J. M. Diego, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** $\Euclid$ \/ is presently mapping the distribution of matter in the Universe in detail via the weak lensing signature of billions of distant galaxies. The weak lensing signal is most prominent around galaxy clusters, and can extend up to distances well beyond their virial radius, thus constraining their total mass.     Near the centre of clusters, where contamination by member galaxies is an issue, the weak lensing data can be complemented with strong lensing data. Strong lensing information can also  diminish the uncertainty due to the mass-sheet degeneracy and provide high-resolution information about the distribution of matter in the centre of clusters. Here we present a joint strong and weak lensing analysis of the $\Euclid$ Early Release Observations  of the cluster Abell 2390  at $z=0.228$ . Thanks to $\Euclid$ 's wide field of view of $0.5 \mathrm{deg}^2$ , combined with its angular resolution in the visible band of $\ang{;;0.13}$ and sampling of $\ang{;;0.1}$ per pixel [ and Acevedo~Barroso (2025)]() , we constrain the density profile in a wide range of radii, 30 kpc $< r <$ 2000 kpc, from the inner region near the brightest cluster galaxy to beyond the virial radius of the cluster.  We find consistency with earlier X-ray results based on assumptions of hydrostatic equilibrium, thus indirectly confirming the nearly relaxed state of this cluster. We also find consistency with previous results based on weak lensing data and ground-based observations of this cluster. From the combined strong+weak lensing profile, we derive the values of the viral mass $M_{200} = (1.48 \pm 0.29)\times10^{15}  \Msun$ ,     and virial radius $r_{200} =(2.05\pm0.13   {\rm Mpc}$ ),     with error bars representing one standard deviation. The profile is well described by an Navarro-Frenk-White model with concentration $c=6.5$ and a small-scale radius of 230 kpc in the  30 kpc $< r <$ 2000 kpc range that is best constrained by strong lensing and weak lensing data.     Abell 2390 is the first of many examples where $\Euclid$ data will play a crucial role in providing masses for clusters. The large coverage provided by $\Euclid$ , combined with the depth of the observations, and high angular resolution, will allow to produce similar results in hundreds of other clusters with rich already available strong lensing data.

</div>

<div id="div_fig1">

<img src="tmp_2507.08545/./Figs/Profile_A2390_MsunPc2_Feb2025_V2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Surface mass density ($\Sigma$) profiles as a function of radius ($r$) centred on the BCG for the three WL clean catalogues. Two NFW profiles are shown in black-grey. The black dotted line is the NFW derived from X-rays assuming hydrostatic equilibrium. The black short-dashed line is an NFW that fits the SL+WL model in the region that is best constrained by the SL+WL data. The light-blue band shows the $1 \sigma$ variability in the \texttt{LensMC} profile when we change the initial guess in the SL+WL optimisation algorithm.  The dark-blue dashed curve is the joint SL+WL solution when we use the raw LensMC in a larger field of view and with a minimal selection of objects (see appendix \ref{sc:lensmc_fullcat}).
   (*fig:A2390_profile*)

</div>
<div id="div_fig2">

<img src="tmp_2507.08545/./Figs/A2390_HST_Euclid_Arcs_CC_Feb2025.png" alt="Fig6" width="100%"/>

**Figure 6. -** Colour composite image (Red=NISP $\JE+\YE+\HE$, Green=HST-ACS-F850LP, Blue=VIS \IE) of the central region of A2390. The circles mark the position of the spectroscopic sample of SL constraints from [Richard, Stark and Ellis (2008)](), [Richard, Claeyssens and Lagattuta (2021)]() colour coded by their spectroscopic redshift (indicated in the upper left). The number next to each small circle is the ID for each family of counterimages.
  We show the critical curves from the SL+WL model
  in cyan at $z_{\rm s}=1.036$ and red at $z_{\rm s}=4.05$.
  The image covers $\ang{;1.73;}\times\ang{;1.18;}$.
   (*fig:A2390_Arcs*)

</div>
<div id="div_fig3">

<img src="tmp_2507.08545/./Figs/NumberShear_G1_G2_Grid_30arcmin_V5_FINAL.png" alt="Fig7" width="100%"/>

**Figure 7. -** Multi-scale grid used for binning the shear measurements in the clean catalogue. The grid is centred in the cluster. _Left_: shows the number of galaxies per square arcminute in each bin when considering only galaxies in the tomographic bins 3--10 described in [Schrabback, Congedo and Gavazzi (2025)]().  The smallest bins in the centre are approximately $\ang{;1;}\times\ang{;1;}$ while in the edges the bins are $\ang{;2.24}\times\ang{;2.24;}$. The full area is  $\ang{;26.85;}\times\ang{;26.85;}$. A similar grid configuration is adopted for the Gaussian functions in the distribution in mass. For the mass grid, the central  $\ang{;4;}\times\ang{;4;}$ region (where the strong lensing constraints are found) is marked with a yellow square. This small region is further divided with an additional set of 123 smaller Gaussians (not shown).
  _Middle and right_: show the observed shear $\gamma_1$ and $\gamma_2$ in the same multi-scale grid, same field of view, and for \texttt{LensMC}. The classic structure of $\gamma_1$ and $\gamma_2$ can be seen (i.e., horizontal and $\pi/4$-oriented quadrupole).
   (*fig:A2390_ShearGrid*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.08545"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand\orcid{#1}$</div>



<div id="title">

# Multi-frequency analysis of the ALMA and VLA high resolution continuum observations of the substructured disc around CI Tau: Preference for sub-mm-sized low-porosity amorphous carbonaceous grains

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.08797-b31b1b.svg)](https://arxiv.org/abs/2507.08797)<mark>Appeared on: 2025-07-14</mark> -  _43 pages, 23 figures, 6 tables. Accepted for publication on A&A_

</div>
<div id="authors">

<mark>F. Zagaria</mark>, et al.

</div>
<div id="abstract">

**Abstract:** We present high angular resolution ( $50 {\rm mas}$ ) and sensitivity Atacama Large Millimeter/submillimeter Array (ALMA) Band 3 ( $3.1 {\rm mm}$ ) and Very Large Array (VLA) Ka Band ( $9.1 {\rm mm}$ ) observations of the multi-ringed disc around the $3 {\rm Myr}$ -old solar-mass star CI Tau. These new data were combined with similar-resolution archival ALMA Band 7 ( $0.9 {\rm mm}$ ) and 6 ( $1.3 {\rm mm}$ ) observations, and new and archival VLA Q ( $7.1 {\rm mm}$ ), Ku ( $2.0 {\rm cm}$ ), X ( $3.0 {\rm cm}$ ), and C Band ( $6.0 {\rm cm}$ ) photometry to study the properties of dust in this system. At wavelengths $\leq3.1 {\rm mm}$ the continuum emission from CI Tau is very extended ( $\geq200 {\rm au}$ ) and highly substructured (with three gaps, four rings, and two additional gap-ring pairs identified by non-parametric visibility modelling). Instead, the VLA Ka Band data are dominated by a centrally peaked bright component, only partially ( $\leq 50\%$ ) due to dust emission, surrounded by a marginally-detected, faint and smooth halo. We fitted the ALMA and VLA Ka Band data together, adopting a physical model that accounts for the effects of dust absorption and scattering. For our fiducial dust composition ("Ricci" opacities), we retrieved a flat maximum grain size distribution across the disc radius, with $a_{\rm max}=(7.1\pm0.8)\times10^{-2} {\rm cm}$ , that we tentatively attributed to fragmentation of fragile dust or bouncing. We tested, for the first time, the dependence of our results on the adopted dust composition model to assess which mixture can best reproduce the observations. We found that the "Ricci" opacities work better than the traditionally adopted "DSHARP" ones, while graphite-rich mixtures perform significantly worse.        We also show that, for our fiducial composition, the data prefer low porosity ( $\leq70\%$ ) grains. This is in contrast with recent claims of highly porous aggregates in younger sources, which we tentatively justified by time-dependent compaction at the fragmentation or bouncing barrier. Our results on composition and porosity are in line with constraints from disc population synthesis models and naturally arise from CI Tau's peculiar spectral behaviour (i.e., the abrupt steepening of its spectral index at wavelengths longer than $3.1 {\rm mm}$ ), making this disc a unique target to characterise the properties of disc solids, thus ideal for deeper cm-wavelength observations and follow-up dust polarisation studies.

</div>

<div id="div_fig1">

<img src="tmp_2507.08797/./image_montage.png" alt="Fig1" width="100%"/>

**Figure 1. -** From top to bottom CI Tau's ALMA Band 7, 6, 3 and VLA Ka Band continuum emission. Left column: \texttt{CLEAN}ed images. Central column: azimuthally averaged surface brightness radial profiles, in violet for \texttt{tclean} images and purple for the best fit \texttt{frank} profiles (a point-source component was subtracted from the 3.1 and $9.1 {\rm mm}$ visibilities before fitting). Right column: \texttt{frank} fit residual images; dotted ellipses mark the location of the dark rings in the \texttt{CLEAN}ed images. The synthesised \texttt{CLEAN} beam is shown as an ellipse in the bottom left corner of each image, and as a segment with FWHM equal to the beam minor axis, in each radial profile subplot. (*fig:1*)

</div>
<div id="div_fig2">

<img src="tmp_2507.08797/./spectral_index.png" alt="Fig3" width="100%"/>

**Figure 3. -** Spectral index radial profiles (solid lines) and their $1\sigma$ uncertainty (shaded areas). The hatched regions mark those locations where ${\rm SNR}\leq5$(left and central panel) and 3 (right panel), for at least one of the emission profiles. The dashed grey lines in each panel display the surface brightness radial profiles combined to determine the spectral index. (*fig:3*)

</div>
<div id="div_fig3">

<img src="tmp_2507.08797/./fit_results/results_comparison.png" alt="Fig17" width="100%"/>

**Figure 17. -** Same as in \autoref{fig:4} but showing a comparison among the best fit results for different compositions: "Ricci (compact)" in blue, "Zubko (BE)" in turquoise, and "DSHARP (default)" in yellow. The highest quality (i.e., lowest $\chi^2$) fits, obtained adopting the "Ricci (compact)" and "Zubko (BE)" optical properties, provide similar marginalised posterior distributions for all the parameters (within a factor of 3 for $a_{\rm max}$ and 2 for $\Sigma_{\rm dust}$). (*fig:A12*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.08797"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

140  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
